In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import statistics
from skimage.measure import regionprops_table

In [ ]:
# from Dieter Baumgartner
# labels the segmented images from Ilastik

def watershed_seeded(Ilastik):
    # Ilastik = Ilastik[300:500, 100:300]
    center = np.where(Ilastik == 1, 255, 0)
    center = np.uint8(center)

    seg = np.where(Ilastik <= 2, 255, 0)
    seg = np.uint8(seg)

    unknown = cv.subtract(seg, center)
    markers = cv.connectedComponents(center)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers[1]+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0

    img2 = np.stack((seg,)*3, axis=2)
    img2 = cv.normalize(img2, None, 255, 0, cv.NORM_MINMAX, cv.CV_8U)   

    # return markers
    return cv.watershed(img2,markers)

In [ ]:
# reads 20 images in mcherry(0001) and cfp(0002) channel that are segmented by Ilastik and label the foreground that contains cells
Ilastik_mcherry = []
labeled_mcherry = []

for n in range(1,21):
    fname = f"example_images/Control Sample 10xdil Image{n:03}0001_seg.tif"
    Ilastik_mcherry.append(np.array(Image.open(fname)))
    labeled_mcherry.append(watershed_seeded(Ilastik_mcherry[n-1]))

Ilastik_cfp = []
labeled_cfp = []

for n in range(1,21):
    fname = f"example_images/Control Sample 10xdil Image{n:03}0002_seg.tif"
    Ilastik_cfp.append(np.array(Image.open(fname)))
    labeled_cfp.append(watershed_seeded(Ilastik_cfp[n-1]))

In [ ]:
# takes one labeled image and gives coordinates of all visible cells

def coordinates(labeled):
    
    props = regionprops_table(labeled, properties=['centroid']) #assuming all cells are centroids
    df = pd.DataFrame(props)
    centroids = df.values.tolist()
    centroids = [[round(x),round(y)] for x,y in centroids]

    return centroids

In [ ]:
# takes one labeled image and gives areas of the visible cells

def cell_area(labeled):
    props = regionprops_table(labeled, properties=['area'])
    df = pd.DataFrame(props)
    areas = df.area.tolist()
    
    return areas

In [ ]:
p = 0 # processes the first image

mcherry = coordinates(labeled_mcherry[p])
area_mcherry = cell_area(labeled_mcherry[p])
avg_cell_mcherry = statistics.median(area_mcherry) # finds the median area of all cells in the image

# repeats for images in cfp channel
cfp = coordinates(labeled_cfp[p])
area_cfp = cell_area(labeled_cfp[p])
cell_cfp = statistics.median(area_cfp)


In [ ]:
# gets the x or y coordinates of the cells in the images
mcherry_x = [x for x,y in mcherry]
mcherry_y = [y for x,y in mcherry]
cfp_x = [x for x,y in cfp]
cfp_y = [y for x,y in cfp]

In [ ]:
d = 5 # [pixel]
# the maximal allowed distance between the candidate pair, i.e. donor and recipient cells

candidates = []
for n in range(1,len(mcherry)+1):
    # puts threshold on the cell size in order to make sure the signal is a cell(not noise or random light spots)
    if 0.8*avg_cell_mcherry <= area_mcherry[n-1] <= 3*avg_cell_mcherry:
        for m in range(1,len(cfp)+1):
            if 0.8*cell_cfp <= area_cfp[m-1] <= 3*cell_cfp:
                if cfp_x[m-1] - d <= mcherry_x[n-1] <= cfp_x[m-1] + d and cfp_y[m-1] - d <= mcherry_y[n-1] <= cfp_y[m-1] + d:
                    candidates.append(mcherry[n-1])

In [ ]:
print(";".join([f"{y},{x}" for x,y in candidates]))